In [1]:
import pandas as pd 

In [2]:
#number of columns

In [3]:
df_jan = pd.read_parquet('data/yellow_taxi_data_jan.parquet')

In [4]:
len(df_jan.columns)

19

In [5]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
#duration in minutes
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration'] = df_jan.duration.apply(lambda x: (x.total_seconds() /60))

df_jan['duration'].std()

42.59435124195458

In [7]:
# percent left after removing outliers

In [8]:
len_df = len(df_jan)

df_jan = df_jan[(df_jan['duration']>=1 )& (df_jan['duration']<=60)]

(len(df_jan)/ len_df)*100

98.1220282212598

In [9]:
#size of dict sparse matrix

In [10]:
categorical = ['PULocationID', 'DOLocationID']
df_jan[categorical] = df_jan[categorical].astype(str)

In [11]:
df_jan_dict = df_jan[categorical].to_dict(orient='records')

In [12]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()

train = dv.fit_transform(df_jan_dict)

In [13]:
train.shape[1]

515

In [14]:
#train a model

In [15]:
from sklearn.linear_model import LinearRegression
X_train  = dv.fit_transform(df_jan_dict)
y_train = df_jan['duration'].values


In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
preds = lr.predict(X_train)

In [19]:
mean_squared_error(preds, y_train, squared=False)

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.64926180044339

In [20]:
# now try validation

def load_dataset(filepath):

    if filepath.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    else:
        df = pd.read_parquet(filepath)
    df['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    # df['duration'] = df['duration'].dt.

    df = df[(df['duration']>=1 ) & (df['duration']<=60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [21]:
val_df = load_dataset('data/yellow_taxi_data_feb.parquet')

In [22]:
val_dict = val_df[categorical].to_dict(orient='records')
val = dv.transform(val_dict)

In [23]:
val_y = val_df['duration'].values

In [24]:
preds = lr.predict(val)

In [25]:
mean_squared_error(val_y,preds,squared=False )

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


11.873335620590654